In [1]:
import os
import sys
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = str(pow(2,40))
import pickle
from glob import glob
from typing import Tuple
from skimage.io import imread
import cv2
import random
import numpy as np
import segmentation_models_pytorch as smp
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
from PIL import Image
#from patchify import patchify, unpatchify
from torch.utils.data import Dataset, DataLoader
import glob
from tqdm.notebook import tqdm
import openslide
import pandas as pd
#from models.HoVerNet.post_proc import process

import os
import time
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.cuda.amp import autocast, GradScaler
from tensorboardX import SummaryWriter
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
import json
import random
from sklearn.metrics import roc_auc_score
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd

import glob
import shutil
import openslide
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import gc

os.environ['CUDA_VISIBLE_DEVICES'] = '0'


In [8]:
input_wsi_path = '../GTEX-1117F-0226.svs'
nuc_map_path = '../nulci.png'

infer_omic = 'humanrna'
assert infer_omic in ['snp', 'ms', 'mirna', 'cnv', 'humanrna', 'methyl']


save_matrix_path = f'../{infer_omic}_scpred.csv'

In [5]:
gene_names = torch.load(f'{infer_omic}_names.pth')

model = timm.create_model('efficientnet_b3.ra2_in1k', pretrained=False)

model.classifier = nn.Sequential(
    nn.Dropout(0.3, inplace=True),
    nn.Linear(in_features=1536, out_features=len(gene_names), bias=True),
)

model.load_state_dict({k.replace('_orig_mod.', ''): v for k, v in torch.load(f'../model_weights/{infer_omic}_model.pth').items()})#, strict=False)

model = model.to('cuda')
model.eval()
print('Done.')

Done.


In [6]:
# Dataset
class OpenSlideInferenceDataset(Dataset):
    def __init__(self, yx_array, transform=None):

        self.yx_array = yx_array
        self.transform = transform

    def __getitem__(self, idx):

        center_y, center_x = self.yx_array[idx]
        #print(center_y, center_x)
        #print(center_y-256, center_y+256, center_x-256, center_x+256)
        img = im_20x[center_y-256:center_y+256, center_x-256:center_x+256]
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']

        return img

    def __len__(self):
        return self.yx_array.shape[0]

In [7]:
transform_val = A.Compose([
    #A.CenterCrop(224, 224),
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #A.Normalize(mean=[0.7343752, 0.734179, 0.7339175], std=[0.1848752, 0.18325073, 0.18443637]),
    ToTensorV2(),
])

In [9]:
input_path = input_wsi_path
slide = openslide.OpenSlide(input_path)
level = slide.properties.get('openslide.objective-power')

width, height = slide.level_dimensions[0]
im_20x = slide.read_region((0, 0), 0, (width, height))
im_20x = np.array(im_20x)[:, :, :3]

if level == '40':
    im_20x = cv2.resize(im_20x, (im_20x.shape[1]//2, im_20x.shape[0]//2))
elif level == '20':
    pass
else:
    raise ValueError('Only WSIs with magnifications of 20 and 40x are supported currently.')

h, w, c = im_20x.shape

binary_mask = cv2.imread(nuc_map_path, cv2.IMREAD_UNCHANGED)
# Find contours
contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Loop through the contours to find bounding box centers
all_yxs = []

for contour in contours:
    # Calculate bounding box
    x, y, w, h = cv2.boundingRect(contour)
    
    # Calculate the center of the bounding box
    cX = x + w // 2
    cY = y + h // 2

    all_yxs.append([cY, cX])

yx_array = np.array(all_yxs)
yx_array[:, 0] = yx_array[:, 0].clip(256, im_20x.shape[0]-256)
yx_array[:, 1] = yx_array[:, 1].clip(256, im_20x.shape[1]-256)

val_dataset = OpenSlideInferenceDataset(yx_array, transform=transform_val)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False, num_workers=16, pin_memory=True, drop_last=False)

all_pres = []

with torch.no_grad():
    for images in tqdm(val_loader):
        images = images.to('cuda')
        with autocast():
            outputs = model(images)

        all_pres.append(outputs.sigmoid().cpu().numpy())

all_pres = np.concatenate(all_pres, axis=0)
all_pres_df = pd.DataFrame(all_pres)
all_pres_df.columns = gene_names
y_x_index_df = pd.DataFrame({
        
                        'x_center': yx_array[:, 1],
                        'y_center': yx_array[:, 0],
                         
                        })
final_df = pd.concat([y_x_index_df, all_pres_df], axis=1)
final_df.T.to_csv(save_matrix_path)

  0%|          | 0/52 [00:00<?, ?it/s]